In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, ReLU, Softmax
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


In [2]:
import cv2
import numpy as np

In [3]:
from tensorflow.keras import backend as K


def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [4]:
SIZE = 256
N = 100

In [5]:
X = np.zeros((N, SIZE, SIZE))
A = np.zeros((N, SIZE, SIZE))
B = np.zeros((N, SIZE, SIZE))
for i in range(1, N + 1):
    X[i - 1] = cv2.imread('data/bnw/' + str(i + 7) + '.jpeg', 0)
    A[i - 1] = cv2.imread('data/a/' + str(i + 7) + '.jpeg', 0)
    B[i - 1] = cv2.imread('data/b/' + str(i + 7) + '.jpeg', 0)



In [6]:
# bins = np.arange(17) * 16
# a_binned = np.digitize(A, bins) - 1
# b_binned = np.digitize(B, bins) - 1
# y_binned = np.stack((a_binned, b_binned)).transpose((1, 2, 3, 0))
bins_list = [0, 64, 96, 112, 120, 128, 136, 144, 160, 192, 256]
num_bins = len(bins_list) - 1
bins = np.array(bins_list)
a_binned = np.digitize(A, bins) - 1
b_binned = np.digitize(B, bins) - 1

In [7]:
# One hot encode A and B
from tensorflow.keras.utils import to_categorical

a_hot = to_categorical(a_binned)
b_hot = to_categorical(b_binned)

In [8]:
X = X.reshape(N, SIZE, SIZE, 1)
y = np.stack((A, B)).transpose((1, 2, 3, 0))

In [9]:
# Normalize the data
X = X / 255
y = y / 255

Following are utility functions for creating a U-net Model

In [10]:
def conv_stack(input_layer, filters):
    conv1 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(input_layer)
    batch_norm1 = BatchNormalization()(conv1)
    relu1 = ReLU()(batch_norm1)

    conv2 = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(relu1)
    batch_norm2 = BatchNormalization()(conv2)
    relu2 = ReLU()(batch_norm2)

    return relu2

In [11]:
def encoder_block(input_layer, filters):
    conv = conv_stack(input_layer, filters)
    max_pool = MaxPooling2D(pool_size=(2, 2))(conv)

    return conv, max_pool

In [12]:
def decoder_block(input_layer, skip_layer, filters):
    up = Conv2DTranspose(filters, 2, strides=2, padding='same')(input_layer)
    conc = Concatenate()([up, skip_layer])
    dec = conv_stack(conc, filters)

    return dec

In [13]:
def get_model(size, init_filters):
    inputs = Input((size, size, 1))

    conv1, max_pool1 = encoder_block(inputs, init_filters)
    conv2, max_pool2 = encoder_block(max_pool1, init_filters * 2)
    conv3, max_pool3 = encoder_block(max_pool2, init_filters * 4)

    middle_block = conv_stack(max_pool3, init_filters * 8)

    decoder1 = decoder_block(middle_block, conv3, init_filters * 4)
    decoder2 = decoder_block(decoder1, conv2, init_filters * 2)
    decoder3 = decoder_block(decoder2, conv1, init_filters)

    # soft = Softmax(axis=1)(decoder3)
    output_a = Conv2D(num_bins, 1, padding='same', activation='softmax')(decoder3)
    # output_b = Conv2D(32, 1, padding='same', activation='softmax')(decoder3)

    model = Model(inputs, output_a)
    return model

In [14]:
model = get_model(SIZE, 64)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 256, 256, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [15]:
weight_a = [276.4182377999916, 10.096705660138996, 2.160336840606263, 1.8126515533081804, 0.3854523724941714,
            0.22361807604925382, 1.040882132359878, 1.6375415188587383, 4.57322774863995, 37.81061231083443]

weight_b = [263.46130653, 6.29587218, 2.67742605, 2.33413422, 0.63087839,
            0.31916905, 0.66331561, 0.56931914, 1.04141953, 10.7601525]
model.compile(optimizer=Adam(0.01), loss=weighted_categorical_crossentropy(weights=weight_a), metrics='accuracy')

In [16]:
callbacks = [
    ModelCheckpoint("aero-color_a.h5", save_best_only=True, save_weights_only=True, monitor='loss')
]

model.fit(X, a_hot, epochs=3, verbose=1, batch_size=8, callbacks=callbacks)

Epoch 1/3
13/13 [==============================] - 143s 11s/step - loss: 0.4468 - accuracy: 0.3375

In [19]:
a_hats = model.predict(X, batch_size=8)

In [20]:
print(np.sum(a_hats[0, :, 0, 0]))
print(np.sum(a_hats[0, 0, :, 0]))
print(np.sum(a_hats[0, 0, 0, :]))

0.0
0.0
1.0


In [21]:
a_hats = np.argmax(a_hats, axis=3)
# b_hats = np.argmax(b_hats, axis=3)

In [22]:
# Real Image
L = X[0].reshape((SIZE, SIZE)) * 255
a = y[0, :, :, 0] * 255
b = y[0, :, :, 1] * 255
lab = np.array([L, a, b]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('001.jpeg', img)

True

In [23]:
# Real Image Binned
L = X[0].reshape((SIZE, SIZE)) * 255
a2 = a_binned[0, :, :]
b2 = b_binned[0, :, :]
a2 = bins[a2] + 0.5 * (bins[a2 + 1] - bins[a2])
b2 = bins[b2] + 0.5 * (bins[b2 + 1] - bins[b2])
lab = np.array([L, a2, b2]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('002.jpeg', img)

True

In [24]:
L = X[0].reshape((SIZE, SIZE)) * 255
a_hat = a_hats[0, :, :]
b_hat = b_binned[0, :, :]
a_hat = bins[a_hat] + 0.5 * (bins[a_hat + 1] - bins[a_hat])
b_hat = bins[b_hat] + 0.5 * (bins[b_hat + 1] - bins[b_hat])
lab = np.array([L, a_hat, b_hat]).transpose((1, 2, 0)).astype('uint8')
img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)
cv2.imwrite('003.jpeg', img)

True

In [26]:
unique, counts = np.unique(a_binned, return_counts=True)
counts = counts / counts.sum()
# weights_a = counts.sum() / counts / num_bins
print(dict(zip(unique, counts)))

{0: 0.00076904296875, 1: 0.008144989013671874, 2: 0.04542404174804687, 3: 0.06859405517578125, 4: 0.2807376098632812, 5: 0.43602035522460936, 6: 0.08103744506835937, 7: 0.06275634765625, 8: 0.015950927734375, 9: 0.000565185546875}
